In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter(action='ignore')

In [18]:
from pycaret.datasets import get_data
data = get_data('electrical_grid')
target = 'stabf'

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


#### Neural Network with SKORCH

In [21]:
import torch.nn as nn
from skorch import NeuralNetClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from skorch.helper import DataFrameTransformer

class MLP(nn.Module):
    def __init__(self,num_inputs=12,num_units_d1=200,num_units_d2=100):
        super(MLP,self).__init__()
        self.dense0 = nn.Linear(num_inputs,num_units_d1)
        self.nonlin = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units_d1,num_units_d2)
        self.output = nn.Linear(num_units_d2,2)
        self.softmax = nn.Softmax(dim=-1)
    def forward(self,X,**kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X

class customNLLLoss(nn.Module):
    criterion = nn.NLLLoss()
    def __init__(self):
        super().__init__()
    def forward(self,logits,target):
         return self.criterion(logits,target.long())

net = NeuralNetClassifier(
    module=MLP,
    criterion=customNLLLoss,
    max_epochs=30,
    lr=0.1,
    batch_size=32,
    train_split=None
)

nn_pipe = Pipeline([
    ('transform', DataFrameTransformer()),
    ('net', net)
])

In [22]:
from pycaret.classification import *

clf1 = setup(
    data=data,
    target=target,
    train_size=0.8,
    fold=5,
    session_id=123,
    log_experiment=True,
    experiment_name='electrical_grid_1',
    silent=True
)

,Description,Value
0,session_id,123
1,Target,stabf
2,Target Type,Binary
3,Label Encoded,"stable: 0, unstable: 1"
4,Original Data,"(10000, 13)"
5,Missing Values,False
6,Numeric Features,12
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [23]:
rf_model = create_model("rf")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9244,0.9796,0.9667,0.9189,0.9422,0.8331,0.8353
1,0.9275,0.9793,0.9549,0.9330,0.9438,0.8417,0.8422
2,0.9225,0.9810,0.9608,0.9211,0.9406,0.8294,0.8309
3,0.9081,0.9738,0.9461,0.9130,0.9293,0.7983,0.7993
4,0.9044,0.9738,0.9471,0.9071,0.9267,0.7894,0.7909
Mean,0.9174,0.9775,0.9551,0.9186,0.9365,0.8184,0.8197
SD,0.0093,0.0031,0.0079,0.0087,0.0071,0.0206,0.0206


In [24]:
skorch_model = create_model(nn_pipe)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8138,0.8750,0.9382,0.8029,0.8653,0.5695,0.5879
1,0.8062,0.8776,0.9667,0.7813,0.8642,0.5393,0.5780
2,0.8094,0.8959,0.8208,0.8729,0.8460,0.5965,0.5984
3,0.7931,0.8495,0.9716,0.7666,0.8570,0.5012,0.5498
4,0.7638,0.8192,0.9726,0.7394,0.8401,0.4185,0.4817
Mean,0.7972,0.8634,0.9340,0.7926,0.8545,0.5250,0.5592
SD,0.0181,0.0266,0.0580,0.0451,0.0099,0.0620,0.0420


In [25]:
best_model = compare_models(include=[skorch_model,rf_model],sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
1,Random Forest Classifier,0.9174,0.9775,0.9551,0.9186,0.9365,0.8184,0.8197,1.2180
0,NeuralNetClassifier,0.8034,0.8755,0.8806,0.8284,0.8508,0.5609,0.5744,8.1100


#### Tuning the Neural Network

In [26]:
import torch.optim as optim

custom_grid = {
    'net__max_epochs': [20,30],
    'net__lr': [0.01,0.05,0.1],
    'net__module__num_units_d1': [50,100,150,200],
    'net__module__num_units_d2': [50,100,150,200],
    'net__optimizer': [optim.Adam,optim.SGD,optim.RMSprop]
}

In [27]:
tuned_skorch_model = tune_model(skorch_model,custom_grid=custom_grid)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7769,0.8424,0.8765,0.7947,0.8336,0.4976,0.5031
1,0.7938,0.8744,0.8333,0.8416,0.8374,0.5554,0.5555
2,0.7819,0.8440,0.9158,0.7805,0.8427,0.4942,0.5113
3,0.7819,0.8564,0.9099,0.7833,0.8419,0.4967,0.5113
4,0.7575,0.8163,0.9598,0.7385,0.8348,0.4071,0.4606
Mean,0.7784,0.8467,0.8991,0.7877,0.8381,0.4902,0.5084
SD,0.0118,0.0190,0.0422,0.0330,0.0037,0.0475,0.0302
